In [61]:
import nomic
import json
nomic.login("nk-2LSEn5FB5h04Dn0pFLuZ9jDw7ZaMbPpYPV88wbKKASc")
#- Scientific programming
import numpy as np
import pandas as pd
from tqdm import tqdm

#- Plotting
import matplotlib.pyplot as plt
import seaborn as sns

#- Data
from nomic import AtlasDataset

#- Processing
from fuzzywuzzy import fuzz
from unidecode import unidecode
import re
import datetime

dataset = AtlasDataset('hivemind/tweets-from-members-of-us-congress-from-all-time-updated-2024-10-08')

df = dataset.maps[0].data.df
embds = dataset.maps[0].embeddings.latent


2024-10-09 13:32:06.000 | INFO     | nomic.dataset:__init__:768 - Loading existing dataset `hivemind/tweets-from-members-of-us-congress-from-all-time-updated-2024-10-08`.
2024-10-09 13:32:06.679 | WARNING  | nomic.data_operations:df:923 - Converting to pandas dataframe. This may materialize a large amount of data into memory.
2024-10-09 13:32:06.682 | INFO     | nomic.data_operations:_download_data:902 - Downloading data
100%|██████████| 129/129 [00:00<00:00, 9453.73it/s]
2024-10-09 13:32:07.061 | INFO     | nomic.data_operations:_load_data:872 - Loading data
100%|██████████| 129/129 [00:02<00:00, 61.93it/s] 
2024-10-09 13:32:13.939 | INFO     | nomic.data_operations:_download_latent:550 - Downloading latent embeddings
100%|██████████| 129/129 [05:10<00:00,  2.41s/it]
2024-10-09 13:37:25.211 | INFO     | nomic.data_operations:latent:527 - Loading latent embeddings
100%|██████████| 129/129 [00:45<00:00,  2.84it/s]


In [113]:
df.head()
congress_name_list= list(df['name'].unique())

In [114]:
n = len(congress_name_list)
all_text = [[] for _ in range(n)]
all_embds = [[] for _ in range(n)]

for i, embd in enumerate(tqdm(embds)):
    congress_name_index = congress_name_list.index(df['name'][i])
    all_text[congress_name_index].append(df['text'][i])
    all_embds[congress_name_index].append(embd)


100%|██████████| 3214913/3214913 [00:43<00:00, 73167.94it/s]


In [121]:
#check if the text is the same
#df[df['name'] == congress_name_list[0]]['text'].tolist()==all_text[0]

True

In [122]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)



Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [123]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True)

<All keys matched successfully>


In [124]:
## GPT-4o generation of questions for bill https://www.govtrack.us/congress/votes/115-2017/s179 

questions = [
    "What do you believe is the best approach to ensure universal access to affordable health care in the U.S.?",
    "How would you address the rising costs of prescription medications in a way that benefits both consumers and pharmaceutical companies?",
    "Do you support expanding Medicare or Medicaid, and why or why not?",
    "What role should the federal government play in regulating health insurance providers?",
    "How do you plan to balance the needs of health care providers with the financial constraints faced by patients, especially low-income families?",
    "What are your views on the role of private versus public health care options in the U.S. system?",
    "How do you plan to ensure that rural and underserved communities have access to quality health care services?",
    "What steps would you take to ensure that individuals with pre-existing conditions maintain access to affordable health care?",
    "Do you believe the U.S. should transition to a single-payer health care system? Why or why not?",
    "How would you address the ongoing debate about health care reform and its intersection with issues like mental health services, reproductive health, and senior care?"
]

In [142]:
name_nan= 'Morgan Griffith'  #'Wesley Hunt' #'James “Jim” Clyburn' #'Earl Blumenauer' #'Charles (Chuck) Edwards'
name_nan = unidecode(name_nan)
name_nan
from fuzzywuzzy import fuzz
highest_ratio_index = np.argmax([fuzz.token_sort_ratio(name_nan, name) for name in congress_name_list])
#fuzz.token_sort_ratio(name_nan, congress_names[highest_ratio_index]),congress_names[highest_ratio_index]
congress_name_list[highest_ratio_index]

'Griffith, Morgan'

In [143]:
i=highest_ratio_index
name=congress_name_list[i]
q=2
question=questions[q]
query_embed=embedding_model.encode(questions[q])

In [144]:
## retirve the tweet that is most similar to the question
retrieve_pos = []
dotprod = []
for j in range(len(all_text[i])):
    dotprod.append(np.sum(all_embds[i][j]*query_embed))
retrieve_pos.append(np.argmax(dotprod))  

In [145]:
rag_retrieved_tweet=all_text[i][retrieve_pos[0]]
rag_retrieved_tweet

'The Senate today voted to overturn President Biden’s vaccine mandate for Medicare and Medicaid providers.\n\nI’m a cosponsor in the House on a resolution that would do the same thing. I don’t say this often, but we should follow the Senate’s lead.'

In [148]:
messages = [
    {"role": "system", "content": "You are posting a Tweet as the account manager for U.S. congressperson /name/.".replace("/name/", name)},
    {"role": "user", "content": "Write a Tweet that addresses /question/. Here is an example Tweet potentially related to the question: /RAG-retireved-Tweet/".replace("/question/", question).replace("/RAG-retireved-Tweet/", rag_retrieved_tweet)},
]

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    messages,
    max_new_tokens=128,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [149]:
print(name, question, outputs[0]["generated_text"][-1]['content'])

Griffith, Morgan Do you support expanding Medicare or Medicaid, and why or why not? "Important question: Should we expand Medicare or Medicaid? As your representative, I want to hear from you! Do you support expanding these programs to ensure more Americans have access to quality healthcare? Share your thoughts and let's work together to improve healthcare for all! #Healthcare #Medicare #Medicaid"
